# Pretrain & PI Finetuning Suite

This notebook orchestrates 20 randomized incremental pretrain runs on non-PI polymers followed by PI-property finetuning with a frozen shared encoder. It combines the continual-task recipes from `dynamic_task_finetuning_demo.ipynb` and `dynamic_task_incremental_finetuning.ipynb`.


## Data Overview

- **Descriptors**: `data/amorphous_polymer_FFDescriptor_20250730.parquet`
- **Non-PI properties**: `data/amorphous_polymer_non_PI_properties_20250730.parquet`
- **PI properties**: `data/amorphous_polymer_PI_properties_20250730.parquet`
- Pretrain tasks: 15 properties (density through thermal_diffusivity) sampled in random order per run
- PI finetune tasks: density, Rg, r2, self-diffusion, Cp, Cv, linear_expansion, refractive_index, tg


In [1]:
import json
import math
import random
import re
from pathlib import Path
from typing import Any

import joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from lightning.pytorch import Trainer
from lightning.pytorch.callbacks import EarlyStopping, ModelCheckpoint
from lightning.pytorch.loggers import CSVLogger, TensorBoardLogger

from foundation_model.data.datamodule import CompoundDataModule
from foundation_model.models.flexible_multi_task_model import FlexibleMultiTaskModel
from foundation_model.models.model_config import OptimizerConfig, RegressionTaskConfig, TaskType


2025-10-31 09:22:25.442 | INFO     | __init__:<module>:34 - Loguru logger initialized for foundation_model package.


In [2]:
DATA_DIR = Path("../data")
DESCRIPTOR_PATH = DATA_DIR / "amorphous_polymer_FFDescriptor_20250730.parquet"
NON_PI_PATH = DATA_DIR / "amorphous_polymer_non_PI_properties_20250730.parquet"
PI_PATH = DATA_DIR / "amorphous_polymer_PI_properties_20250730.parquet"
SCALER_PATH = DATA_DIR / "amorphous_polymer_properties_scaler_20250730.pkl.z"

USE_NORMALIZED_TARGETS = False
ALL_TASK_NAMES = [
    "density",
    "Rg",
    "r2",
    # "self-diffusion",
    # "Cp",
    # "Cv",
    # "bulk_modulus",
    # "volume_expansion",
    # "linear_expansion",
    # "static_dielectric_const",
    # "dielectric_const_dc",
    # "refractive_index",
    # "tg",
    # "thermal_conductivity",
    # "thermal_diffusivity",
]
PI_TASK_NAMES = [
    "density",
    "Rg",
    "r2",
    # "self-diffusion",
    # "Cp",
    # "Cv",
    # "linear_expansion",
    # "refractive_index",
    # "tg",
]
TARGET_COLUMNS = {
    name: f"{name}{'(normalized)' if USE_NORMALIZED_TARGETS else ''}"
    for name in ALL_TASK_NAMES
}

SHARED_BLOCK_DIMS = [190, 256, 128]
HEAD_HIDDEN = 64
ARTIFACT_ROOT = Path("../artifacts/polymers_pretrain_finetune_runs")
ARTIFACT_ROOT.mkdir(parents=True, exist_ok=True)

NUM_PRETRAIN_RUNS = 2  # Number of different task orders to try
PRETRAIN_MAX_EPOCHS = 10  # Max epochs for pretraining
FINETUNE_MAX_EPOCHS = 10  # Max epochs for finetuning each task
BATCH_SIZE = 256
NUM_WORKERS = 0
LOG_EVERY_N_STEPS = 50
RANDOM_SEED_BASE = 1729

PRETRAIN_SAMPLE = None  # Set to an int for smoke tests
PI_SAMPLE = None  # Set to an int for smoke tests

PROPERTY_SCALERS: dict[str, Any] = {}


In [3]:
descriptor_df = pd.read_parquet(DESCRIPTOR_PATH)
non_pi_df = pd.read_parquet(NON_PI_PATH)
pi_df = pd.read_parquet(PI_PATH)

if USE_NORMALIZED_TARGETS:
    if not SCALER_PATH.exists():
        raise FileNotFoundError(f"Missing scaler file: {SCALER_PATH}")
    PROPERTY_SCALERS = joblib.load(SCALER_PATH)
    missing_scalers = [name for name in ALL_TASK_NAMES if name not in PROPERTY_SCALERS]
    if missing_scalers:
        raise KeyError(f"Scaler missing entries for: {missing_scalers}")
else:
    PROPERTY_SCALERS = {}

missing_non_pi = [TARGET_COLUMNS[name] for name in ALL_TASK_NAMES if TARGET_COLUMNS[name] not in non_pi_df.columns]
if missing_non_pi:
    raise KeyError(f"Non-PI table missing columns: {missing_non_pi}")

missing_pi = [TARGET_COLUMNS[name] for name in PI_TASK_NAMES if TARGET_COLUMNS[name] not in pi_df.columns]
if missing_pi:
    raise KeyError(f"PI table missing columns: {missing_pi}")

common_non_pi_index = descriptor_df.index.intersection(non_pi_df.index)
pretrain_features = descriptor_df.loc[common_non_pi_index]
pretrain_targets = non_pi_df.loc[common_non_pi_index, [TARGET_COLUMNS[name] for name in ALL_TASK_NAMES]]

if PRETRAIN_SAMPLE is not None and PRETRAIN_SAMPLE < len(pretrain_features):
    pretrain_features = pretrain_features.sample(n=PRETRAIN_SAMPLE, random_state=42)
    pretrain_targets = pretrain_targets.loc[pretrain_features.index]

common_pi_index = descriptor_df.index.intersection(pi_df.index)
pi_features = descriptor_df.loc[common_pi_index]
pi_targets = pi_df.loc[common_pi_index, [TARGET_COLUMNS[name] for name in PI_TASK_NAMES]]

if PI_SAMPLE is not None and PI_SAMPLE < len(pi_features):
    pi_features = pi_features.sample(n=PI_SAMPLE, random_state=13)
    pi_targets = pi_targets.loc[pi_features.index]

print(f"Pretrain feature matrix: {pretrain_features.shape}")
print(f"Pretrain target matrix: {pretrain_targets.shape}")
print(f"PI feature matrix: {pi_features.shape}")
print(f"PI target matrix: {pi_targets.shape}")


Pretrain feature matrix: (71725, 190)
Pretrain target matrix: (71725, 3)
PI feature matrix: (1083, 190)
PI target matrix: (1083, 3)


## Helper Utilities


In [4]:
def safe_slug(name: str) -> str:
    slug = re.sub(r"[^a-z0-9]+", "_", name.lower()).strip("_")
    return slug or "task"

def maybe_inverse_transform(property_name: str, values: np.ndarray) -> np.ndarray:
    if not USE_NORMALIZED_TARGETS:
        return values
    scaler = PROPERTY_SCALERS.get(property_name)
    if scaler is None:
        raise KeyError(f"Scaler not found for property '{property_name}'")
    reshaped = values.reshape(-1, 1)
    restored = scaler.inverse_transform(reshaped)
    return np.asarray(restored).reshape(-1)

def build_regression_task(name: str, column: str) -> RegressionTaskConfig:
    return RegressionTaskConfig(
        name=name,
        data_column=column,
        dims=[SHARED_BLOCK_DIMS[-1], HEAD_HIDDEN, 1],
        norm=True,
        residual=False,
    )

def make_pretrain_task_configs(task_names: list[str]) -> list[RegressionTaskConfig]:
    return [build_regression_task(name, TARGET_COLUMNS[name]) for name in task_names]

def make_pi_task_config(task_name: str) -> RegressionTaskConfig:
    return build_regression_task(task_name, TARGET_COLUMNS[task_name])

def build_pretrain_datamodule(task_names: list[str], *, batch_size: int = BATCH_SIZE) -> CompoundDataModule:
    stage_targets = pretrain_targets.loc[:, [TARGET_COLUMNS[name] for name in task_names]]
    return CompoundDataModule(
        formula_desc_source=pretrain_features,
        attributes_source=stage_targets,
        task_configs=make_pretrain_task_configs(task_names),
        batch_size=batch_size,
        num_workers=NUM_WORKERS,
    )

def build_pi_datamodule(task_name: str, *, batch_size: int = BATCH_SIZE) -> CompoundDataModule:
    target_frame = pi_targets.loc[:, [TARGET_COLUMNS[task_name]]]
    task_config = make_pi_task_config(task_name)
    return CompoundDataModule(
        formula_desc_source=pi_features,
        attributes_source=target_frame,
        task_configs=[task_config],
        batch_size=batch_size,
        num_workers=NUM_WORKERS,
    )

def plot_test_predictions(
    *,
    model: FlexibleMultiTaskModel,
    datamodule: CompoundDataModule,
    phase: str,
    run_id: int,
    stage_num: int,
    stage_tasks: list[str],
    new_task_name: str,
    output_dir: Path | str,
) -> None:
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)
    metrics_path = output_dir / "metrics.json"
    predictions_path = output_dir / "predictions.parquet"
    task_order_path = output_dir / "tasks.txt"
    task_order_path.write_text(" -> ".join(stage_tasks) + "", encoding="utf-8")

    if torch.cuda.is_available():
        device = "cuda"
    elif hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
        device = "mps"
    else:
        device = "cpu"

    datamodule.setup(stage="test")
    test_loader = datamodule.test_dataloader()
    if test_loader is None:
        raise RuntimeError(f"{phase} stage {stage_num} has no test dataloader")

    original_device = next(model.parameters()).device
    was_training = model.training
    model = model.to(device)
    model.eval()

    aggregated: dict[str, dict[str, list[torch.Tensor]]] = {}
    prediction_rows: list[dict[str, float | int | str]] = []
    per_task_counts: dict[str, int] = {}

    with torch.no_grad():
        for batch in test_loader:
            x, y_dict, mask_dict, t_sequences = batch
            x = x.to(device)
            preds = model(x, t_sequences)

            for name, pred_tensor in preds.items():
                if name not in y_dict:
                    continue

                target_tensor = y_dict[name]
                mask_tensor = mask_dict.get(name)

                if isinstance(target_tensor, list):
                    target_flat = torch.cat([t.detach().cpu().reshape(-1) for t in target_tensor])
                else:
                    target_flat = target_tensor.detach().cpu().reshape(-1)

                pred_flat = pred_tensor.detach().cpu().reshape(-1)

                if mask_tensor is not None:
                    if isinstance(mask_tensor, list):
                        mask_flat = torch.cat([m.detach().cpu().reshape(-1) for m in mask_tensor])
                    else:
                        mask_flat = mask_tensor.detach().cpu().reshape(-1)
                    mask_flat = mask_flat.bool()
                    target_flat = target_flat[mask_flat]
                    pred_flat = pred_flat[mask_flat]

                if target_flat.numel() == 0:
                    continue

                target_np = target_flat.numpy()
                pred_np = pred_flat.numpy()
                target_np = maybe_inverse_transform(name, target_np)
                pred_np = maybe_inverse_transform(name, pred_np)

                entry = aggregated.setdefault(name, {"preds": [], "targets": []})
                entry["preds"].append(torch.from_numpy(pred_np.astype(np.float32)))
                entry["targets"].append(torch.from_numpy(target_np.astype(np.float32)))

                start_idx = per_task_counts.get(name, 0)
                for offset, (actual_val, pred_val) in enumerate(zip(target_np.tolist(), pred_np.tolist())):
                    prediction_rows.append(
                        {
                            "run": run_id,
                            "phase": phase,
                            "stage": stage_num,
                            "task": name,
                            "sample_index": start_idx + offset,
                            "actual": actual_val,
                            "predicted": pred_val,
                        }
                    )
                per_task_counts[name] = start_idx + len(target_np)

    if not aggregated:
        print(f"No predictions to log for run {run_id} stage {stage_num} ({phase}).")
        model.to(original_device)
        if was_training:
            model.train()
        return

    ordered_items = [(name, aggregated[name]) for name in stage_tasks if name in aggregated]

    metrics: dict[str, dict[str, float | int | None]] = {}
    num_tasks = len(ordered_items)
    cols = 2 if num_tasks > 1 else 1
    rows = math.ceil(num_tasks / cols)
    fig, axes = plt.subplots(rows, cols, figsize=(cols * 4.5, rows * 4.5))
    if hasattr(axes, "flat"):
        axes_list = list(axes.flat)
    else:
        axes_list = [axes]

    for ax, (name, data) in zip(axes_list, ordered_items):
        preds = torch.cat(data["preds"]).numpy()
        targets = torch.cat(data["targets"]).numpy()
        diff = preds - targets
        mae = float(np.mean(np.abs(diff)))
        mse = float(np.mean(diff ** 2))
        rmse = float(np.sqrt(np.mean(diff ** 2)))
        ss_tot = float(np.sum((targets - np.mean(targets)) ** 2))
        ss_res = float(np.sum(diff ** 2))
        r2_value = 1.0 - ss_res / ss_tot if ss_tot > 0 else None

        metrics[name] = {
            "samples": int(targets.size),
            "mae": mae,
            "mse": mse,
            "rmse": rmse,
            "r2": r2_value,
        }

        lo = float(min(preds.min(), targets.min()))
        hi = float(max(preds.max(), targets.max()))
        buffer = 0.05 * (hi - lo) if hi > lo else 0.1
        lo -= buffer
        hi += buffer

        ax.scatter(targets, preds, s=12, alpha=0.6, edgecolors="none")
        ax.plot([lo, hi], [lo, hi], "--", color="tab:red", linewidth=1)
        annotation = rf"MAE: {mae:.3f} $R^2$: {r2_value:.3f}" if r2_value is not None else f"MAE: {mae:.3f}"
        ax.text(
            0.05,
            0.95,
            annotation,
            transform=ax.transAxes,
            fontsize=10,
            verticalalignment="top",
            bbox=dict(boxstyle="round,pad=0.3", facecolor="white", alpha=0.6),
        )
        ax.set_xlim(lo, hi)
        ax.set_ylim(lo, hi)
        ax.set_xlabel("Actual")
        ax.set_ylabel("Predicted")
        ax.set_title(f"{phase.title()} Stage {stage_num}: {name}")
        ax.grid(alpha=0.2)
        ax.set_aspect("equal", adjustable="box")

    for ax in axes_list[len(ordered_items):]:
        ax.axis("off")

    fig.tight_layout()
    fig.savefig(output_dir / f"{phase}_stage{stage_num:02d}_overview.png", dpi=180)
    plt.close(fig)

    for name, data in ordered_items:
        preds = torch.cat(data["preds"]).numpy()
        targets = torch.cat(data["targets"]).numpy()
        lo = float(min(preds.min(), targets.min()))
        hi = float(max(preds.max(), targets.max()))
        buffer = 0.05 * (hi - lo) if hi > lo else 0.1
        lo -= buffer
        hi += buffer

        fig_single, ax_single = plt.subplots(figsize=(5, 5))
        ax_single.scatter(targets, preds, s=12, alpha=0.6, edgecolors="none")
        ax_single.plot([lo, hi], [lo, hi], "--", color="tab:red", linewidth=1)
        ax_single.set_xlim(lo, hi)
        ax_single.set_ylim(lo, hi)
        ax_single.set_xlabel("Actual")
        ax_single.set_ylabel("Predicted")
        ax_single.set_title(f"{phase.title()} Stage {stage_num}: {name}")
        ax_single.grid(alpha=0.2)
        ax_single.set_aspect("equal", adjustable="box")
        fig_single.tight_layout()
        ax_file = output_dir / f"{safe_slug(name)}_pred.png"
        fig_single.savefig(ax_file, dpi=180)
        plt.close(fig_single)

    metrics_payload = {
        "run_id": run_id,
        "phase": phase,
        "stage": stage_num,
        "new_task": new_task_name,
        "task_sequence": list(stage_tasks),
        "metrics": metrics,
    }

    if prediction_rows:
        pd.DataFrame(prediction_rows).to_parquet(predictions_path, index=False)
        print(f"Saved predictions to {predictions_path}")

    with open(metrics_path, "w", encoding="utf-8") as f:
        json.dump(metrics_payload, f, indent=2)
    print(f"Saved metrics to {metrics_path}")

    model.to(original_device)
    if was_training:
        model.train()


In [5]:
torch.serialization.add_safe_globals([RegressionTaskConfig, TaskType, OptimizerConfig])


## Pretrain & Finetune Workflow


In [6]:
experiment_records: list[dict] = []

for run_idx in range(1, NUM_PRETRAIN_RUNS + 1):
    rng = random.Random(RANDOM_SEED_BASE + run_idx)
    task_sequence = rng.sample(ALL_TASK_NAMES, k=len(ALL_TASK_NAMES))
    run_label = f"run{run_idx:02d}"
    print(f"""
====================
Starting {run_label}
Task order: {task_sequence}
====================
""")

    run_root = ARTIFACT_ROOT / run_label
    run_root.mkdir(parents=True, exist_ok=True)

    previous_checkpoint: str | None = None
    pretrain_stage_records: list[dict] = []

    for stage_idx, task_name in enumerate(task_sequence, start=1):
        stage_tasks = task_sequence[:stage_idx]
        datamodule = build_pretrain_datamodule(stage_tasks)
        task_configs = make_pretrain_task_configs(stage_tasks)

        if previous_checkpoint is None:
            model = FlexibleMultiTaskModel(
                shared_block_dims=SHARED_BLOCK_DIMS,
                task_configs=task_configs,
                enable_learnable_loss_balancer=True,
                shared_block_optimizer=OptimizerConfig(lr=1e-2),
            )
        else:
            model = FlexibleMultiTaskModel.load_from_checkpoint(
                checkpoint_path=previous_checkpoint,
                strict=False,
                enable_learnable_loss_balancer=True,
            )
            existing = set(model.task_heads.keys())
            new_configs = [cfg for cfg in task_configs if cfg.name not in existing]
            if new_configs:
                model.add_task(*new_configs)

        stage_dir = run_root / f"pretrain_stage{stage_idx:02d}_{safe_slug(task_name)}"
        stage_dir.mkdir(parents=True, exist_ok=True)

        checkpoint_cb = ModelCheckpoint(
            dirpath=stage_dir / "checkpoints",
            filename=f"{safe_slug(task_name)}-{{epoch:02d}}-{{val_final_loss:.4f}}",
            monitor="val_final_loss",
            mode="min",
            save_top_k=1,
        )
        early_stopping = EarlyStopping(monitor="val_final_loss", mode="min", patience=10)
        csv_logger = CSVLogger(save_dir=stage_dir / "logs", name="csv")
        tensorboard_logger = TensorBoardLogger(save_dir=stage_dir / "logs", name="tensorboard")

        trainer = Trainer(
            max_epochs=PRETRAIN_MAX_EPOCHS,
            accelerator="auto",
            devices="auto",
            callbacks=[checkpoint_cb, early_stopping],
            logger=[csv_logger, tensorboard_logger],
            log_every_n_steps=LOG_EVERY_N_STEPS,
        )

        trainer.fit(model, datamodule=datamodule)
        best_model_path = checkpoint_cb.best_model_path
        print(f"Run {run_label} stage {stage_idx}: best checkpoint -> {best_model_path}")

        if best_model_path:
            state = torch.load(best_model_path, map_location="cpu", weights_only=True)
            state_dict = state.get("state_dict", state)
            model.load_state_dict(state_dict)
            previous_checkpoint = best_model_path
        else:
            print("Warning: no best checkpoint captured; using current weights.")

        prediction_dir = stage_dir / "prediction"
        plot_test_predictions(
            model=model,
            datamodule=datamodule,
            phase="pretrain",
            run_id=run_idx,
            stage_num=stage_idx,
            stage_tasks=stage_tasks,
            new_task_name=task_name,
            output_dir=prediction_dir,
        )

        pretrain_stage_records.append(
            {
                "stage": stage_idx,
                "task_name": task_name,
                "task_sequence": list(stage_tasks),
                "checkpoint": best_model_path,
                "stage_dir": stage_dir,
            }
        )

    if previous_checkpoint is None:
        raise RuntimeError(f"Run {run_label} produced no pretrain checkpoint; cannot finetune.")

    finetune_records: list[dict] = []
    for finetune_idx, task_name in enumerate(PI_TASK_NAMES, start=1):
        finetune_model = FlexibleMultiTaskModel.load_from_checkpoint(
            checkpoint_path=previous_checkpoint,
            strict=False,
            enable_learnable_loss_balancer=True,
            freeze_shared_encoder=True,
        )
        active_tasks = list(finetune_model.task_heads.keys())
        if active_tasks:
            finetune_model.remove_tasks(*active_tasks)

        task_config = make_pi_task_config(task_name)
        finetune_model.add_task(task_config)

        datamodule = build_pi_datamodule(task_name)

        stage_dir = run_root / f"finetune_stage{finetune_idx:02d}_{safe_slug(task_name)}"
        stage_dir.mkdir(parents=True, exist_ok=True)

        checkpoint_cb = ModelCheckpoint(
            dirpath=stage_dir / "checkpoints",
            filename=f"{safe_slug(task_name)}-{{epoch:02d}}-{{val_final_loss:.4f}}",
            monitor="val_final_loss",
            mode="min",
            save_top_k=1,
        )
        early_stopping = EarlyStopping(monitor="val_final_loss", mode="min", patience=10)
        csv_logger = CSVLogger(save_dir=stage_dir / "logs", name="csv")
        tensorboard_logger = TensorBoardLogger(save_dir=stage_dir / "logs", name="tensorboard")

        trainer = Trainer(
            max_epochs=FINETUNE_MAX_EPOCHS,
            accelerator="auto",
            devices="auto",
            callbacks=[checkpoint_cb, early_stopping],
            logger=[csv_logger, tensorboard_logger],
            log_every_n_steps=LOG_EVERY_N_STEPS,
        )

        trainer.fit(finetune_model, datamodule=datamodule)
        best_model_path = checkpoint_cb.best_model_path
        print(f"Run {run_label} finetune {task_name}: best checkpoint -> {best_model_path}")

        if best_model_path:
            state = torch.load(best_model_path, map_location="cpu", weights_only=True)
            state_dict = state.get("state_dict", state)
            finetune_model.load_state_dict(state_dict)
        else:
            print("Warning: finetune stage missing checkpoint; using current weights.")

        prediction_dir = stage_dir / "prediction"
        plot_test_predictions(
            model=finetune_model,
            datamodule=datamodule,
            phase="finetune",
            run_id=run_idx,
            stage_num=finetune_idx,
            stage_tasks=[task_name],
            new_task_name=task_name,
            output_dir=prediction_dir,
        )

        finetune_records.append(
            {
                "stage": finetune_idx,
                "task_name": task_name,
                "checkpoint": best_model_path,
                "stage_dir": stage_dir,
            }
        )

    experiment_records.append(
        {
            "run": run_label,
            "task_sequence": task_sequence,
            "pretrain": pretrain_stage_records,
            "pretrain_checkpoint": previous_checkpoint,
            "finetune": finetune_records,
        }
    )

print("Completed all pretrain + finetune runs.")


2025-10-31 09:22:57.642 | INFO     | datamodule:__init__:160 - Initializing CompoundDataModule...
2025-10-31 09:22:57.643 | INFO     | datamodule:__init__:192 - --- Loading Data ---
2025-10-31 09:22:57.643 | INFO     | datamodule:_load_data:432 - Using provided pd.DataFrame for 'formula_desc' data.
2025-10-31 09:22:57.657 | INFO     | datamodule:_load_data:439 - Successfully loaded 'formula_desc'. Shape: (71725, 190)
2025-10-31 09:22:57.658 | INFO     | datamodule:__init__:197 - Initial loaded formula_df length: 71725
2025-10-31 09:22:57.675 | INFO     | datamodule:__init__:204 - Formula_df length after initial dropna: 71725. This index is now the master reference.
2025-10-31 09:22:57.675 | INFO     | datamodule:_load_data:432 - Using provided pd.DataFrame for 'attributes' data.
2025-10-31 09:22:57.675 | INFO     | datamodule:_load_data:439 - Successfully loaded 'attributes'. Shape: (71725, 1)
2025-10-31 09:22:57.675 | INFO     | datamodule:__init__:218 - Initial loaded attributes_df l


Starting run01
Task order: ['density', 'Rg', 'r2']



2025-10-31 09:22:57.845 | INFO     | dataset:__init__:99 - [train_dataset] Initializing CompoundDataset...
2025-10-31 09:22:57.850 | INFO     | dataset:__init__:133 - [train_dataset] Final x_formula shape: torch.Size([57379, 190])
2025-10-31 09:22:57.850 | INFO     | dataset:__init__:148 - [train_dataset] Processing enabled task 'density' (type: REGRESSION)
2025-10-31 09:22:57.942 | INFO     | dataset:__init__:334 - [train_dataset] CompoundDataset initialization complete. Processed 1 enabled tasks.
2025-10-31 09:22:57.945 | INFO     | datamodule:_build_fit_datasets:483 - Creating val_dataset with 7173 samples.
2025-10-31 09:22:57.950 | INFO     | dataset:__init__:99 - [val_dataset] Initializing CompoundDataset...
2025-10-31 09:22:57.951 | INFO     | dataset:__init__:133 - [val_dataset] Final x_formula shape: torch.Size([7173, 190])
2025-10-31 09:22:57.952 | INFO     | dataset:__init__:148 - [val_dataset] Processing enabled task 'density' (type: REGRESSION)
2025-10-31 09:22:57.964 | INF

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/Users/liuchang/projects/foundation_model/.venv/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:433: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


...

Epoch 9: 100%|██████████| 4/4 [00:00<00:00, 72.39it/s, v_num=0, train_final_loss_step=529.0, val_final_loss=152.0, train_final_loss_epoch=904.0]


2025-10-31 09:26:00.254 | INFO     | datamodule:setup:551 - --- Setting up DataModule for stage: test ---
2025-10-31 09:26:00.254 | INFO     | datamodule:setup:561 - Total samples available before splitting (from attributes_df index): 1078
2025-10-31 09:26:00.255 | INFO     | datamodule:setup:591 - Data split strategy: Performing random train/val/test splits based on full_idx (derived from attributes_df).
2025-10-31 09:26:00.255 | INFO     | datamodule:setup:594 - Test split ratio: 0.1, Validation split ratio (of non-test): 0.1
2025-10-31 09:26:00.255 | INFO     | datamodule:setup:606 - Split full data (1078) into train_val (970) and test (108) using seed 24.
2025-10-31 09:26:00.256 | INFO     | datamodule:setup:632 - Split train_val (970) into train (862) and val (108) using seed 42, effective_val_split 0.111.
2025-10-31 09:26:00.256 | INFO     | datamodule:setup:658 - Final dataset sizes after splitting: Train=862, Validation=108, Test=108
2025-10-31 09:26:00.256 | INFO     | datamod

Run run02 finetune r2: best checkpoint -> /Users/liuchang/projects/foundation_model/artifacts/polymers_pretrain_finetune_runs/run02/finetune_stage03_r2/checkpoints/r2-epoch=09-val_final_loss=151.9535.ckpt


2025-10-31 09:26:00.257 | INFO     | dataset:__init__:334 - [test_dataset] CompoundDataset initialization complete. Processed 1 enabled tasks.
2025-10-31 09:26:00.258 | INFO     | datamodule:setup:703 - --- DataModule setup for stage 'test' complete ---


Saved predictions to ../artifacts/polymers_pretrain_finetune_runs/run02/finetune_stage03_r2/prediction/predictions.parquet
Saved metrics to ../artifacts/polymers_pretrain_finetune_runs/run02/finetune_stage03_r2/prediction/metrics.json
Completed all pretrain + finetune runs.


## Run Summary


In [7]:
print(f"Recorded {len(experiment_records)} runs.")
for record in experiment_records:
    print(record["run"], "pretrain stages:", len(record["pretrain"]), "finetune stages:", len(record["finetune"]))


Recorded 2 runs.
run01 pretrain stages: 3 finetune stages: 3
run02 pretrain stages: 3 finetune stages: 3
